In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torch
import numpy as np
import torch.nn.functional as F
import torchvision.models as models
import scipy
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision

torch.cuda.empty_cache()
rpath = 'r_data'
fpath = 'validation'
transform_train = transforms.Compose([ transforms.ToTensor(),transforms.Resize((224,224))])

train_dataset = ImageFolder(root=rpath, transform=transform_train)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=4,shuffle=True)

test_dataset = ImageFolder(root=fpath, transform=transform_train)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle = False)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = models.vgg16(pretrained=True)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(),lr=0.0000015)
save_path='weight'
net.classifier._modules['6'] = nn.Linear(4096, 4)
net.to(device)
print(net)

res=[]
for epoch in range(3):  # loop over the dataset multiple times
    running_loss = 0.0
    l = []

    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs= net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 40 == 0:
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / (i+1)))
            l.append([epoch + 1, i + 1, running_loss / (i+1)])
            running_loss = 0.0
    res.append(l)


    torch.save(net.state_dict(), save_path)
    torch.cuda.empty_cache()

print('Finished Training')

class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

net.load_state_dict(torch.load(save_path))
net.to(device)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

accuracy_sum=0
lab = ["burgerking",'cu','gs25','lotteria']
for i in range(4):
    temp = 100 * class_correct[i] / class_total[i]
    print('Accuracy of %5s : %2d %%' % (
        lab[i], temp))
    accuracy_sum+=temp
print('Accuracy average: ', accuracy_sum/4)



